In [1]:
# Include libraries which may use in implementation
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib.image import imread
import glob
import cv2
from sklearn.preprocessing import OneHotEncoder
#from PIL import Image as img

#from keras.models import load_model
#from keras.models import model_from_json
%matplotlib inline


In [2]:
train_x=[]
train_y = []
test_x = []
test_y = []

path = 'C:\\Users\\Tauseef Chahal\\DeepLearning_Code\\Deep\\Assignment2\\Task3_Data\\'
for i in range(10):
    for filename in glob.glob(path + 'train/' +str(i)+'/*.png'):
        im = imread(filename)
        train_x.append(im)
        train_y.append(i)
for i in range(10):
    for filename in glob.glob(path + 'test/'+str(i)+'/*.png'):
        im = imread(filename)
        test_x.append(im)
        test_y.append(i)

train_x = np.array(train_x)
train_y = np.array(train_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

In [3]:
print(train_x.shape)

(60000, 28, 28)


In [4]:
enc = OneHotEncoder(handle_unknown='ignore')

enc.fit(train_y.reshape(train_y.shape[0],1))
train_y_hat = enc.transform(train_y.reshape(train_y.shape[0],1))
train_y_hat.shape

(60000, 10)

In [5]:
import scipy.io #loading data from a mat file
data = scipy.io.loadmat('filters.mat') 
filters = data['filters'] 
filters = np.array(filters)
filters.shape

(17, 17, 36)

In [22]:
def zero_pad(X, pad):

    X_pad = np.pad(X, ((0, 0), (pad, pad), (pad, pad)), 'constant', constant_values=0)
    return X_pad

In [23]:
def conv_single_step(img_patch, filters):
    x_w = np.multiply(img_patch,filters)
    return np.sum(x_w)

In [36]:
def conv_forward(img, filters, stride=1, pad=0):
    
    (n_H_prev, n_W_prev, n_C_prev) = img.shape
    ( f, n_C_prev, n_C) = filters.shape
    
    n_H = int((n_H_prev - f + 2 *pad)/stride)+1
    n_W = int((n_W_prev -f + 2* pad)/stride)+1
    
    A_prev_pad = zero_pad(img, pad)
    
    Z = np.zeros(( n_H, n_W, n_C))
    for i in range(n_H_prev):
        a_prev_pad = A_prev_pad[i]
        for h in range(n_H):                          
             for w in range(n_W):                       
                for c in range(n_C):                   
                     # Find the corners of the current "slice" (≈4 lines)
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f
                    # Use the corners to define the (3D) slice of a_prev_pad (See Hint above the cell). (≈1 line)
                    a_slice_prev = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end ]
                    Z[i, h, w, c] = conv_single_step(a_slice_prev, W[...,c])

In [37]:
def convolution(image, filt, bias, s=1):
    (n_f, n_c_f, f, _) = filt.shape # filter dimensions
    n_c, in_dim, _ = image.shape # image dimensions
    
    out_dim = int((in_dim - f)/s)+1 # calculate output dimensions
    
    # ensure that the filter dimensions match the dimensions of the input image
    assert n_c == n_c_f, "Dimensions of filter must match dimensions of input image"
    
    out = np.zeros((n_f,out_dim,out_dim)) # create the matrix to hold the values of the convolution operation
    
    # convolve each filter over the image
    for curr_f in range(n_f):
        curr_y = out_y = 0
        # move filter vertically across the image
        while curr_y + f <= in_dim:
            curr_x = out_x = 0
            # move filter horizontally across the image 
            while curr_x + f <= in_dim:
                # perform the convolution operation and add the bias
                out[curr_f, out_y, out_x] = np.sum(filt[curr_f] * image[:,curr_y:curr_y+f, curr_x:curr_x+f]) + bias[curr_f]
                curr_x += s
                out_x += 1
            curr_y += s
            out_y += 1
        
    return out

ValueError: too many values to unpack (expected 3)